In [347]:
# importing pandas and numpy
import pandas as pd
import numpy as np
import collections
import itertools as it
import time
import re
support=50
confidence = 70

In [348]:
data = pd.read_csv('/content/drive/My Drive/association-rule-test-data.txt', sep="\t", header=None)
data.head(2)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100
0,Up,Up,Down,Up,Down,Up,Up,Down,Down,Up,Up,Up,Down,Down,Down,Down,Up,Up,Down,Up,Up,Up,Up,Down,Down,Down,Down,Down,Down,Up,Down,Up,Up,Down,Down,Down,Down,Up,Up,Up,...,Down,Up,Down,Up,Up,Down,Up,Down,Up,Down,Up,Down,Up,Down,Down,Up,Down,Down,Up,Down,Down,Up,Up,Up,Down,Down,Up,Down,Down,Down,Up,Up,Down,Up,Down,Down,Down,Up,Down,ALL
1,Up,Down,Up,Down,Up,Down,Down,Down,Down,Up,Up,Up,Down,Up,Up,Down,Down,Up,Down,Down,Up,Up,Up,Up,Down,Up,Up,Up,Up,Down,Down,Up,Up,Down,Up,Up,Up,Down,Down,Up,...,Down,Down,Down,Down,Up,Down,Up,Down,Up,Down,Up,Up,Down,Up,Down,Down,Up,Down,Up,Up,Down,Down,Up,Down,Up,Up,Down,Up,Down,Down,Up,Up,Down,Down,Down,Down,Down,Up,Up,ALL


In [349]:
# Appending G1 to the data
for i in range(len(data.columns)-1):
  data[:][i] = "G"+str(i+1)+ "_"+data.iloc[:][i]
data.head(2)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100
0,G1_Up,G2_Up,G3_Down,G4_Up,G5_Down,G6_Up,G7_Up,G8_Down,G9_Down,G10_Up,G11_Up,G12_Up,G13_Down,G14_Down,G15_Down,G16_Down,G17_Up,G18_Up,G19_Down,G20_Up,G21_Up,G22_Up,G23_Up,G24_Down,G25_Down,G26_Down,G27_Down,G28_Down,G29_Down,G30_Up,G31_Down,G32_Up,G33_Up,G34_Down,G35_Down,G36_Down,G37_Down,G38_Up,G39_Up,G40_Up,...,G62_Down,G63_Up,G64_Down,G65_Up,G66_Up,G67_Down,G68_Up,G69_Down,G70_Up,G71_Down,G72_Up,G73_Down,G74_Up,G75_Down,G76_Down,G77_Up,G78_Down,G79_Down,G80_Up,G81_Down,G82_Down,G83_Up,G84_Up,G85_Up,G86_Down,G87_Down,G88_Up,G89_Down,G90_Down,G91_Down,G92_Up,G93_Up,G94_Down,G95_Up,G96_Down,G97_Down,G98_Down,G99_Up,G100_Down,ALL
1,G1_Up,G2_Down,G3_Up,G4_Down,G5_Up,G6_Down,G7_Down,G8_Down,G9_Down,G10_Up,G11_Up,G12_Up,G13_Down,G14_Up,G15_Up,G16_Down,G17_Down,G18_Up,G19_Down,G20_Down,G21_Up,G22_Up,G23_Up,G24_Up,G25_Down,G26_Up,G27_Up,G28_Up,G29_Up,G30_Down,G31_Down,G32_Up,G33_Up,G34_Down,G35_Up,G36_Up,G37_Up,G38_Down,G39_Down,G40_Up,...,G62_Down,G63_Down,G64_Down,G65_Down,G66_Up,G67_Down,G68_Up,G69_Down,G70_Up,G71_Down,G72_Up,G73_Up,G74_Down,G75_Up,G76_Down,G77_Down,G78_Up,G79_Down,G80_Up,G81_Up,G82_Down,G83_Down,G84_Up,G85_Down,G86_Up,G87_Up,G88_Down,G89_Up,G90_Down,G91_Down,G92_Up,G93_Up,G94_Down,G95_Down,G96_Down,G97_Down,G98_Down,G99_Up,G100_Up,ALL


In [350]:
freq_item_set = {}
last_freq_item_sets = {}

In [351]:
c={}
for i in range(len(data.columns)):
  c.update(collections.Counter(data.iloc[:,i].values))

In [352]:
start_time = time.time()
from collections import defaultdict
freq_item=[]
dict2={}
for key in c:
  if c[key] >= support:
    freq_item.append(tuple([key]))
    dict2[tuple([key])] = c[key]
print(len(freq_item))

dict1=defaultdict(list)
for key in dict2.keys():
  for i in range(len(data)):
    if key in data.loc[i,:].values:
      dict1[key].append(i)
temp = list(dict1.keys())
for key in temp:
  if key not in dict2.keys():
    del dict1[key]
print("--- %s seconds ---" % (time.time() - start_time))
freq_item_set.update(dict2)

109
--- 1.4293136596679688 seconds ---


In [353]:
print("DICT1",dict1)
print("DICT2",dict2)


DICT1 defaultdict(<class 'list'>, {('G1_Up',): [0, 1, 4, 5, 6, 8, 10, 12, 13, 14, 15, 17, 18, 19, 20, 21, 22, 23, 24, 28, 30, 31, 32, 33, 35, 36, 37, 38, 39, 40, 43, 45, 46, 47, 48, 49, 53, 55, 56, 58, 60, 61, 62, 63, 64, 66, 67, 69, 70, 71, 72, 73, 75, 76, 77, 80, 81, 82, 83, 84, 86, 88, 89, 90, 92, 93, 94, 96, 97], ('G2_Down',): [1, 2, 3, 5, 7, 8, 9, 10, 12, 13, 15, 16, 17, 18, 19, 20, 22, 24, 26, 30, 31, 33, 34, 35, 36, 38, 41, 42, 44, 45, 46, 48, 49, 50, 52, 53, 54, 55, 56, 57, 58, 60, 61, 62, 65, 67, 68, 69, 70, 72, 73, 74, 75, 78, 79, 80, 82, 83, 85, 86, 88, 90, 93, 94, 96, 97], ('G3_Down',): [0, 3, 4, 5, 6, 11, 12, 14, 17, 18, 20, 21, 22, 25, 27, 28, 29, 31, 34, 38, 39, 45, 48, 49, 50, 51, 53, 57, 58, 62, 64, 65, 67, 69, 70, 72, 73, 74, 75, 77, 80, 81, 83, 85, 86, 87, 92, 95, 96, 99], ('G3_Up',): [1, 2, 7, 8, 9, 10, 13, 15, 16, 19, 23, 24, 26, 30, 32, 33, 35, 36, 37, 40, 41, 42, 43, 44, 46, 47, 52, 54, 55, 56, 59, 60, 61, 63, 66, 68, 71, 76, 78, 79, 82, 84, 88, 89, 90, 91, 93, 9

In [354]:

def generate_combinations(f,length):
  start_time = time.time()
  comb_freq_items = []
  for i in it.combinations(f,2):
    temp=tuple(sorted(set(i[0]+i[1])))
    if len(temp) == length:
      comb_freq_items.append(temp)
  comb_freq_items=set(comb_freq_items)
  return comb_freq_items

In [355]:

def func_dict(dict1,dict_count,items_freq):
  start_time = time.time()  
  d1=defaultdict(list)  # gives rows of each n-1 len combo
  d2={}     # gives count of each n-1 len combo
  for i in items_freq:
    count = 0
    if i[:-1] in dict1.keys():
      col_num1 = re.findall("\d+", i[-1])
      if col_num1 ==[]:
        col_num = len(data.columns)
      else:
        col_num = int(col_num1[0])
      for j in dict1[i[:-1]]:
        if i[-1] == data[col_num-1][j]:
          d1[i].append(j)
          count = count + 1
      if count >= support:
        d2[i] = count
  freq_item_set.update(d2)
  temp = list(d1.keys())
  for key in temp:
    if key not in d2.keys():
      del d1[key]
  return (d1,d2)


In [356]:
%%time
i=1
support_count_dict={}
print("number of length-1 frequent itemsets:",len(freq_item_set))
total_length=len(freq_item_set)
while(freq_item):
  i=i+1
  comb_freq_items=generate_combinations(freq_item,i)
  freq_item=[]
  (dict1,dict2) = func_dict(dict1,dict2,comb_freq_items)
  freq_item = list(dict2.keys())
  print("number of length-",str(i), "frequent itemsets:",len(freq_item))
  total_length += len(freq_item)
  if freq_item != []:
    last_freq_item_sets = freq_item

print("number of all lengths frequent itemsets:",total_length)

number of length-1 frequent itemsets: 109
number of length- 2 frequent itemsets: 63
number of length- 3 frequent itemsets: 2
number of length- 4 frequent itemsets: 0
number of all lengths frequent itemsets: 174
CPU times: user 2 s, sys: 4.94 ms, total: 2 s
Wall time: 2.01 s


In [357]:
def subs(l):
    if l == []:
        return [[]]

    x = subs(l[1:])

    return x + [[l[0]] + y for y in x]

In [358]:
def func_conf(head,body):
  if (freq_item_set[tuple(sorted(head+body))]/freq_item_set[tuple(sorted(head))]) >= confidence/100:
    return (head,body)
  else:
    return ([],[])

In [359]:
head_list = []
body_list = []
rule_list = []
head_dict = dict1=defaultdict(list)
body_dict = dict1=defaultdict(list)
for k in freq_item_set:
  rule_items=list(k)
  for li1 in subs(rule_items):
    if li1 == [] or li1 == rule_items:
      continue
    else:
      body = [i for i in li1 + rule_items if i not in li1 or i not in rule_items]
      head = li1
      (head,body) = func_conf(head,body)
      if head != [] and body != []:
        head_list.append(head)
        body_list.append(body)
        rule_list.append(head + ["->"] + body)
        head_dict[tuple(head)].append(head + ["->"] + body)
        body_dict[tuple(body)].append(head + ["->"] + body)

In [360]:
len(rule_list)

117

In [361]:
class asso_rule:
  def template1(entity,cardinality,items):
    HEAD = []
    count = 0
    if entity == "HEAD": 
      if cardinality == "ANY":
        for i in head_list:
          if any(item in items for item in i) == True:
            temp = head_dict[tuple(i)]
            if temp not in HEAD:
              count = count+len(temp)
              HEAD.append(temp)
        return HEAD,count
      elif cardinality == "NONE":
        for i in head_list:
          if any(item in items for item in i) == False:
            temp = head_dict[tuple(i)]
            if temp not in HEAD:
              count = count+len(temp)
              HEAD.append(temp)
        return HEAD,count
      else:
        c = 0
        for n in head_list:
          count = 0 
          for m in items: 
            if m in n: 
              count = count + 1
              if count == cardinality:
                temp = head_dict[tuple(n)]
                if temp not in HEAD:
                  c = c + len(temp)
                  HEAD.append(temp)
                break  
        return HEAD,c
    elif entity == "BODY":
      if cardinality == "ANY":
        for i in body_list:
          if any(item in items for item in i) == True:
            temp = body_dict[tuple(i)]
            if temp not in HEAD:
              count = count+len(temp)
              HEAD.append(temp)
        return HEAD,count
      elif cardinality == "NONE":
        for i in body_list:
          if any(item in items for item in i) == False:
            temp = body_dict[tuple(i)]
            if temp not in HEAD:
              count = count+len(temp)
              HEAD.append(temp)
        return HEAD,count
      else:
        c = 0
        for n in body_list:
          count = 0 
          for m in items: 
            if m in n: 
              count = count + 1
              if count == cardinality:
                temp = body_dict[tuple(n)]
                if temp not in HEAD:
                  c = c+len(temp)
                  HEAD.append(temp)
                break  
        return (HEAD,c) 
    elif entity == "RULE":
      if cardinality == "ANY":
        for i in rule_list:
          if any(item in items for item in i) == True:
            count = count+1
            HEAD.append([i])
        return HEAD,count
      elif cardinality == "NONE":
        for i in rule_list:
          if any(item in items for item in i) == False:
            count = count+1
            HEAD.append([i])
        return HEAD,count
      else:
        c = 0
        for n in rule_list:
          count = 0 
          for m in items: 
            if m in n: 
              count = count + 1
              if count == cardinality:
                HEAD.append([n])
                c = c + 1
                break  
        return (HEAD,c)


  def template2(entity,cardinality):
    HEAD=[]
    count = 0
    if entity=="HEAD":
      for i in head_list:
        if len(i)>= cardinality:
            temp = head_dict[tuple(i)]
            if temp not in HEAD:
              count = count + len(temp)
              HEAD.append(temp)
    elif entity=="BODY":
      for i in body_list:
        if len(i)>= cardinality:
            temp = body_dict[tuple(i)]
            if temp not in HEAD:
              count = count + len(temp)
              HEAD.append(temp)
    elif entity=="RULE":
      for i in rule_list:
        if len(i)>= cardinality+1:
          HEAD.append([i])
          count = count + 1
    return HEAD,count
  
  def template3(clause,entity1, *l):
    con = re.findall("\D+", clause)[0]
    if len(l)== 5:
      cardinality1 = l[0] 
      item1 = l[1] 
      entity2 = l[2] 
      cardinality2 = l[3] 
      item2 = l[4]
      c1,cnt1 = asso_rule.template1(entity1,cardinality1,item1)
      c2,cnt2 = asso_rule.template1(entity2,cardinality2,item2) 
      clause1_list = []
      clause2_list = []
      for i in c1:
        clause1_list.extend(i)
      for i in c2:
        clause2_list.extend(i)
      clause1_tuple = tuple(tuple(sub) for sub in clause1_list)
      clause2_tuple = tuple(tuple(sub) for sub in clause2_list)
      if con == 'or':
        final_tuple = set(clause1_tuple) | set(clause2_tuple)
      elif con == 'and':
        final_tuple = set(clause1_tuple) & set(clause2_tuple)
    
    
    elif len(l)==4:
      cardinality1 = l[0] 
      item1 = l[1] 
      entity2 = l[2] 
      cardinality2 = l[3]
      c1,cnt1 = asso_rule.template1(entity1,cardinality1,item1)
      c2,cnt2 = asso_rule.template2(entity2,cardinality2) 
      clause1_list = []
      clause2_list = []
      for i in c1:
        clause1_list.extend(i)
      for i in c2:
        clause2_list.extend(i)
      clause1_tuple = tuple(tuple(sub) for sub in clause1_list)
      clause2_tuple = tuple(tuple(sub) for sub in clause2_list)
      if con == 'or':
        final_tuple = set(clause1_tuple) | set(clause2_tuple)
      elif con == 'and':
        final_tuple = set(clause1_tuple) & set(clause2_tuple)
      

    elif len(l)==3:
      cardinality1 = l[0] 
      entity2 = l[1] 
      cardinality2 = l[2]
      c1,cnt1 = asso_rule.template2(entity1,cardinality1)
      c2,cnt2 = asso_rule.template2(entity2,cardinality2) 
      clause1_list = []
      clause2_list = []
      for i in c1:
        clause1_list.extend(i)
      for i in c2:
        clause2_list.extend(i)
      clause1_tuple = tuple(tuple(sub) for sub in clause1_list)
      clause2_tuple = tuple(tuple(sub) for sub in clause2_list)
      if con == 'or':
        final_tuple = set(clause1_tuple) | set(clause2_tuple)
      elif con == 'and':
        final_tuple = set(clause1_tuple) & set(clause2_tuple)
    return final_tuple,len(final_tuple)






In [362]:
print(asso_rule.template1("RULE", "ANY", ['G59_Up'])[1])
print(asso_rule.template1("RULE", "NONE", ['G59_Up'])[1])
print(asso_rule.template1("RULE", 1, ['G59_Up', 'G10_Down'])[1])
print(asso_rule.template1("HEAD", "ANY", ['G59_Up'])[1])
print(asso_rule.template1("HEAD", "NONE", ['G59_Up'])[1])
print(asso_rule.template1("HEAD", 1, ['G59_Up', 'G10_Down'])[1])
print(asso_rule.template1("BODY", "ANY", ['G59_Up'])[1])
print(asso_rule.template1("BODY", "NONE", ['G59_Up'])[1])
print(asso_rule.template1("BODY", 1, ['G59_Up', 'G10_Down'])[1])

26
91
40
9
108
17
17
100
24


In [363]:
print(asso_rule.template2("RULE", 3)[1])
print(asso_rule.template2("HEAD", 2)[1])
print(asso_rule.template2("BODY", 1)[1])

9
6
117


In [364]:
print(asso_rule.template3("1or1", "HEAD", "ANY", ['G10_Down'], "BODY", 1, ['G59_Up'])[1])
print(asso_rule.template3("1and1", "HEAD", "ANY", ['G10_Down'], "BODY", 1, ['G59_Up'])[1])
print(asso_rule.template3("1or2", "HEAD", "ANY", ['G10_Down'], "BODY", 2)[1])
print(asso_rule.template3("1and2", "HEAD", "ANY", ['G10_Down'], "BODY", 2)[1])
print(asso_rule.template3("2or2", "HEAD", 1, "BODY", 2)[1])
print(asso_rule.template3("2and2", "HEAD", 1, "BODY", 2)[1])

24
1
11
0
117
3


In [365]:
asso_rule.template3("2and2", "HEAD", 1, "BODY", 2)[0]

{('G72_Up', '->', 'G59_Up', 'G82_Down'),
 ('G82_Down', '->', 'G59_Up', 'G72_Up'),
 ('G96_Down', '->', 'G59_Up', 'G72_Up')}